In [ ]:
import os
import tensorflow as tf
import import_ipynb

In [ ]:
from datetime import datetime
from tensorflow.keras.optimizers import SGD
from tensorflow.keras import layers, Sequential
from data_preprocessing import train_ds, test_ds
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, TensorBoard, EarlyStopping

In [ ]:
num_classes = 7

In [ ]:
model = Sequential([
    layers.experimental.preprocessing.Normalization(),
    
    layers.Conv2D(32, (3, 3), padding="same", activation="relu"),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Dropout(0.2),
    
    layers.Conv2D(64, (3, 3), padding="same", activation="relu"),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Dropout(0.2),
    
    layers.Conv2D(128, (3, 3), padding="same", activation="relu"),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Dropout(0.2),
    
    layers.Flatten(),
    layers.Dense(256, activation="relu"),
    layers.Dropout(0.2),
    
    layers.Dense(num_classes, activation="softmax")
])

In [ ]:
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)

In [ ]:
model.compile(optimizer=sgd, loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=["accuracy"])

In [ ]:
os.makedirs("models", exist_ok=True)
start_time = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

In [ ]:
model.fit(train_ds, epochs=90,
          callbacks=[
             ReduceLROnPlateau(monitor="val_loss", factor=0.2, patience=10, verbose=1, mode="auto", min_lr=1e-05),
#              ModelCheckpoint("models/%s.h5" % (start_time), monitor="val_loss", save_best_only=True, mode="min", verbose=1),
             TensorBoard(log_dir="logs/%s" % (start_time)),
             EarlyStopping(monitor="val_loss", patience=3)
         ],
         use_multiprocessing=True,
         workers=5,
         max_queue_size=5)

model.summary()